In [2]:
!pip install -q torch


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [145]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os


In [217]:
# Load the dataset
# data = pd.read_csv('CommissionsDataSet.csv')

data = pd.read_csv('synthetic_dataSet.csv')

# Separate features and target variable
X = data.drop(columns=['commission'])
y = data['commission']

# Encode categorical variables
categorical_features = ['policy_type', 'region', 'add_ons']
encoder = LabelEncoder()
for feature in categorical_features:
    X[feature] = encoder.fit_transform(X[feature])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.head())

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

      policy_value  policy_type  region  add_ons
9254  42379.481451            1       0        0
1561  16079.359975            1       2        0
1670  15471.463927            1       3        1
6087  22244.603839            1       3        0
6669  22748.781380            0       2        1


In [218]:
# Define the neural network model
class CommissionPredictor(nn.Module):
    def __init__(self, input_dim,l2_reg):
        super(CommissionPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Increased number of neurons
        self.fc2 = nn.Linear(128, 64)          # Added another hidden layer
        self.fc3 = nn.Linear(64, 32)           # Added another hidden layer
        self.fc4 = nn.Linear(32, 1)            # Output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [230]:
# Define hyperparameters
input_dim = X_train.shape[1]  # Number of input features
lr = 0.01# Learning rate
num_epochs = 50
batch_size = 30
l2_reg = 0.001  # L2 regularization strength

In [231]:
# Initialize model, loss function, and optimizer
model = CommissionPredictor(input_dim, l2_reg=l2_reg)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [232]:
# Training loop
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        optimizer.zero_grad()
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50], Loss: 11300.3428
Epoch [2/50], Loss: 12380.7520
Epoch [3/50], Loss: 12224.2256
Epoch [4/50], Loss: 12023.8574
Epoch [5/50], Loss: 11942.7695
Epoch [6/50], Loss: 11906.9219
Epoch [7/50], Loss: 11883.1426
Epoch [8/50], Loss: 11887.5762
Epoch [9/50], Loss: 11882.2676
Epoch [10/50], Loss: 11884.3584
Epoch [11/50], Loss: 11879.6221
Epoch [12/50], Loss: 11838.2920
Epoch [13/50], Loss: 11889.5439
Epoch [14/50], Loss: 11890.7822
Epoch [15/50], Loss: 11881.0479
Epoch [16/50], Loss: 11900.1924
Epoch [17/50], Loss: 11913.3799
Epoch [18/50], Loss: 11928.7285
Epoch [19/50], Loss: 11925.6123
Epoch [20/50], Loss: 11934.1904
Epoch [21/50], Loss: 11941.2764
Epoch [22/50], Loss: 11955.0469
Epoch [23/50], Loss: 11961.8467
Epoch [24/50], Loss: 11990.0371
Epoch [25/50], Loss: 12013.3027
Epoch [26/50], Loss: 12039.0107
Epoch [27/50], Loss: 12063.0527
Epoch [28/50], Loss: 12080.6240
Epoch [29/50], Loss: 12091.8760
Epoch [30/50], Loss: 12089.0693
Epoch [31/50], Loss: 12093.8760
Epoch [32/50], Lo

In [193]:
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions.squeeze(), y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 10953.5869


In [179]:

model_filename = "commission_predictor_model.pth"
# Get the current working directory
current_dir = os.getcwd()

# Construct the full file path by joining the current working directory with the file name
model_path = os.path.join(current_dir, model_filename)

# Save the model to the specified file path
torch.save(model.state_dict(), model_path)